<a href="https://colab.research.google.com/github/Pavun-KumarCH/Research-Notebooks/blob/main/Semantic_Search_VDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title requirements
%pip install --q pinecone sentence_transformers torch tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
import time
import json
import torch
from torch import nn
from tqdm.auto import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

In [ ]:
from google.colab import userdata
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [ ]:
#@title Load the Dataset
dataset = load_dataset('quora', split = 'train[240000:290000]', trust_remote_code = True)

In [ ]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [ ]:
## Filter out the questions
questions = []
for record in dataset['questions']:
  questions.extend(record['text'])
question = list(set(questions)) # Unique questions
print('\n'.join(questions[:10]),"\n")
print("-"* 100,"\n") # end line

print(f"Number of questions: {len(questions)}")

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2? 

---------------------------------------------------------------------------------------------------- 

Number of questions: 100000


**Note:**


> "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this we are using CPUs. So, you might notice some code cells taking a little longer to run.






* We are using all-MiniLM-L6-v2 sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [ ]:
## Set up the device
device = "cuda" if torch.cuda.is_available() else "cpu"
if device != "cuda":
  print("Sorry no Cuda.")
model = SentenceTransformer('all-MiniLM-L6-v2', device = device)

Sorry no Cuda.


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "Which city i sthe Most populated in the world ?"
query_embedding = model.encode(query)
query_embedding.shape

(384,)

In [ ]:
def create_dlai_index_name(index_name):
    openai_key = ""
    try:
        # For Google Colab
        from google.colab import userdata
        openai_key = userdata.get("OPENAI_API_KEY")
    except ImportError:
        # For Jupyter or other environments
        openai_key = os.getenv("OPENAI_API_KEY")

    # Ensure openai_key is not empty
    if not openai_key:
        raise ValueError("OpenAI API key is missing.")

    return f'{index_name}-{openai_key[-36:].lower().replace("_", "-")}'


In [ ]:
#@title Setup Pinecone
pinecone = Pinecone(api_key = os.environ['PINECONE_API_KEY'])

INDEX_NAME = create_dlai_index_name("avdb")

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)

pinecone.create_index(name = INDEX_NAME,
                      dimension = model.get_sentence_embedding_dimension(),
                      metric = 'cosine',
                      spec = ServerlessSpec(cloud = "aws", region = "us-east-1"))

index = pinecone.Index(INDEX_NAME)
print(index)

avdb-vpopyske-1jrsrfdve1xcz1o4wbhanimodua


In [ ]:
#@title ## Create Embeddings and Upsert(Uploading) to Pinecone

batch_size = 200
vector_limit = 10000

questions = question[:vector_limit]

for i in tqdm(range(0, len(questions), batch_size)):
  # Find end of the batch
  i_end = min(i+batch_size, len(questions))

  # create ID's batch
  ids = [str(x) for x in range(i, i_end)]

  # Create Metadata batch
  metadatas = [{"text" : text} for text in questions[i : i_end]]

  # Create Embeddings
  xc = model.encode(questions[i : i_end])

  # Create records list for upsert
  records = zip(ids, xc, metadatas)

  # Upsert to Pinecone
  index.upsert(vectors = records)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

In [ ]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k = 10, vector = embedding, include_metadata = True, include_values = False) # Here we want actual text not embedding values

  for result in results['matches']:
    print(f"{round(result['score'], 2)} : {result['metadata']['text']}")

In [ ]:
#@title Run your Query
run_query("which city has the highest population in the world?")

0.7 : Which is the most beautiful city in world?
0.66 : Which is the best city in the world to travel?
0.64 : Which is the largest country in the world?
0.64 : Which city has the most museums per capita?
0.62 : How is the world's population estimated?
0.58 : What is the biggest country?
0.58 : What is the most dangerous city in USA?
0.56 : Which is the best city in India?
0.55 : About 50% of the world population is concentrated between the latitudes of?
0.54 : Which one is the most powerfull country in world?


In [ ]:
query = 'How to Attract Girls ?'
run_query(query)

0.72 : How do I seduce girls?
0.71 : How can I make myself more attractive to girls?
0.65 : What should be done to impress a girl?
0.61 : How can I be more confident, especially when approaching girls?
0.57 : What is the best way to impress a girl in a meeting?
0.56 : What are the best ways to seduce someone?
0.56 : How do I get to win a girl's heart?
0.54 : How can I stay away from girls?
0.53 : What is the best way to make a girl happy?
0.52 : How do I stay away from girls?
